In [1]:
from tensorflow import keras
import tensorflow
import kerastuner 

/opt/anaconda3/envs/Diss/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
from tensorflow.keras.layers import (
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    MaxPooling2D, 
    BatchNormalization
)

import random
import tensorflow as tf

In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_breast_cancer()
X = iris.data[:, :]  # we only take the first two features.
y = iris.target

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X,y)

In [5]:
#importing StandardScaler
from sklearn.preprocessing import StandardScaler
#creating object
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [6]:
METRICS = ['accuracy']

In [19]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', min_delta=0, patience=5, verbose=0,
    mode='max', baseline=None, restore_best_weights=True
)

In [20]:
def build_model(hp):
    
    model = keras.Sequential()
    
    for i in range(hp.Choice('num_layers', [1,2,3])):
        model.add(Dense(units=hp.Choice('layer_' + str(i) + '_width',[8, 16, 32, 64, 128]), \
                        kernel_initializer = hp.Choice('kernel_' + str(i), ['glorot_uniform']), \
                        activation= hp.Choice('activation_' + str(i), ['relu', 'tanh'])))
        model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Choice('dropout_'+ str(i),[0.0,0.1,0.2, 0.5])))
        
    ## Add different optimizers
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1.0, 0.1, 0.01, 0.001])),\
                  loss='binary_crossentropy',\
                  metrics=METRICS)
    return model

In [31]:
import kerastuner
import numpy as np
from sklearn import model_selection
import time

class CVTuner(kerastuner.engine.tuner.Tuner):
    def run_trial(self, trial, x, y, executions = 3, *args, **kwargs):
        
        kwargs['batch_size'] = trial.hyperparameters.Choice('batch_size', [8, 16, 64, 128])
        
        exec_store = []
        for exe in range(executions):
            cv = model_selection.KFold(3)
            val_acc = []
            for train_indices, test_indices in cv.split(x):
                x_train, x_val = x[train_indices], x[test_indices]
                y_train, y_val = y[train_indices], y[test_indices]
                print(x_train, y_train)
                
                model = self.hypermodel.build(trial.hyperparameters)
                model.fit(x_train, y_train, \
                          batch_size=kwargs['batch_size'], \
                          epochs=kwargs['epochs'], \
                          validation_data = (x_val, y_val), callbacks = [early_stop])

                val_acc.append(model.evaluate(x_val, y_val)[1])
            ## Mean or max??
            exec_store.append(np.mean(val_acc))
        
        self.oracle.update_trial(trial.trial_id, {'val_accuracy': np.mean(exec_store)})
        self.save_model(trial.trial_id, model)

tuner = CVTuner(
  hypermodel=build_model,
  oracle=kerastuner.oracles.BayesianOptimization(
    objective=kerastuner.Objective("val_accuracy", direction="max"),
    max_trials=30), \
    directory='Tuning',\
    project_name= str(time.time()))

## add a callback 
tuner.search(x_train, y_train,\
             epochs=60, executions = 3)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
num_layers        |1                 |?                 
layer_0_width     |16                |?                 
kernel_0          |glorot_uniform    |?                 
activation_0      |relu              |?                 
dropout_0         |0.1               |?                 
learning_rate     |0.001             |?                 

[[ 1.09404954 -1.37896118  0.9642964  ... -0.4616729  -1.7970857
  -1.38759638]
 [-0.00956305 -0.80459238 -0.0792854  ... -0.48075481 -1.23257821
  -0.92789302]
 [-0.56281009 -0.31767891 -0.55997562 ... -0.77843257  0.22746523
  -0.72461537]
 ...
 [-0.98062687  1.4550643  -0.97974503 ... -0.79827775  0.80316692
  -0.14053458]
 [-0.74434427 -0.71713705 -0.77069485 ... -0.73538378 -0.0891707
   0.19698302]
 [-0.00380006 -0.72895534 -0.07135735 ... -0.59723077 -0.29066629
  -0.64078389]] [1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 0 0 1 0
 0 1 1 1

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7fe442af5290>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/Diss/lib/python3.7/weakref.py", line 358, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt


36/36 [==============================] - 1s 6ms/step - loss: 0.7089 - accuracy: 0.5444 - val_loss: 0.4865 - val_accuracy: 0.8099
Epoch 2/60
36/36 [==============================] - 0s 2ms/step - loss: 0.4953 - accuracy: 0.8041 - val_loss: 0.3492 - val_accuracy: 0.9085
Epoch 3/60
36/36 [==============================] - 0s 2ms/step - loss: 0.3911 - accuracy: 0.8191 - val_loss: 0.2798 - val_accuracy: 0.9366
Epoch 4/60
36/36 [==============================] - 0s 2ms/step - loss: 0.2882 - accuracy: 0.9181 - val_loss: 0.2334 - val_accuracy: 0.9437
Epoch 5/60
36/36 [==============================] - 0s 2ms/step - loss: 0.2689 - accuracy: 0.9161 - val_loss: 0.1996 - val_accuracy: 0.9507
Epoch 6/60
36/36 [==============================] - 0s 2ms/step - loss: 0.2206 - accuracy: 0.9211 - val_loss: 0.1749 - val_accuracy: 0.9507
Epoch 7/60
36/36 [==============================] - 0s 2ms/step - loss: 0.2102 - accuracy: 0.9228 - val_loss: 0.1604 - val_accuracy: 0.9577
Epoch 8/60
36/36 [=============

KeyboardInterrupt: 

In [22]:
tuner.results_summary(1)

Results summary
Results in Tuning/1627681856.6119978
Showing 1 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 2
layer_0_width: 8
kernel_0: glorot_uniform
activation_0: tanh
dropout_0: 0.0
learning_rate: 0.01
batch_size: 8
layer_1_width: 128
kernel_1: glorot_uniform
activation_1: tanh
dropout_1: 0.2
layer_2_width: 64
kernel_2: glorot_uniform
activation_2: tanh
dropout_2: 0.1
Score: 0.9827856024106344


In [23]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=10)[0]
print(best_hps.values)

{'num_layers': 2, 'layer_0_width': 8, 'kernel_0': 'glorot_uniform', 'activation_0': 'tanh', 'dropout_0': 0.0, 'learning_rate': 0.01, 'batch_size': 8, 'layer_1_width': 128, 'kernel_1': 'glorot_uniform', 'activation_1': 'tanh', 'dropout_1': 0.2, 'layer_2_width': 64, 'kernel_2': 'glorot_uniform', 'activation_2': 'tanh', 'dropout_2': 0.1}


In [24]:
# Build the model with the optimal hyperparameters
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=150, validation_split=0.2)

Epoch 1/150
11/11 [==============================] - 1s 17ms/step - loss: 0.2394 - accuracy: 0.9192 - val_loss: 0.1617 - val_accuracy: 0.9535
Epoch 2/150
11/11 [==============================] - 0s 5ms/step - loss: 0.0999 - accuracy: 0.9641 - val_loss: 0.1664 - val_accuracy: 0.9302
Epoch 3/150
11/11 [==============================] - 0s 5ms/step - loss: 0.0563 - accuracy: 0.9861 - val_loss: 0.1856 - val_accuracy: 0.9419
Epoch 4/150
11/11 [==============================] - 0s 5ms/step - loss: 0.0495 - accuracy: 0.9761 - val_loss: 0.1554 - val_accuracy: 0.9535
Epoch 5/150
11/11 [==============================] - 0s 5ms/step - loss: 0.0343 - accuracy: 0.9903 - val_loss: 0.1352 - val_accuracy: 0.9651
Epoch 6/150
11/11 [==============================] - 0s 4ms/step - loss: 0.0389 - accuracy: 0.9846 - val_loss: 0.1476 - val_accuracy: 0.9535
Epoch 7/150
11/11 [==============================] - 0s 5ms/step - loss: 0.0524 - accuracy: 0.9761 - val_loss: 0.1178 - val_accuracy: 0.9651
Epoch 8/150


In [25]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 22


In [26]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(x_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/22
11/11 [==============================] - 1s 17ms/step - loss: 0.3456 - accuracy: 0.8380 - val_loss: 0.1977 - val_accuracy: 0.9302
Epoch 2/22
11/11 [==============================] - 0s 5ms/step - loss: 0.1384 - accuracy: 0.9549 - val_loss: 0.2054 - val_accuracy: 0.9302
Epoch 3/22
11/11 [==============================] - 0s 5ms/step - loss: 0.1211 - accuracy: 0.9693 - val_loss: 0.2417 - val_accuracy: 0.9070
Epoch 4/22
11/11 [==============================] - 0s 4ms/step - loss: 0.0460 - accuracy: 0.9882 - val_loss: 0.2541 - val_accuracy: 0.9302
Epoch 5/22
11/11 [==============================] - 0s 4ms/step - loss: 0.0614 - accuracy: 0.9780 - val_loss: 0.2301 - val_accuracy: 0.9302
Epoch 6/22
11/11 [==============================] - 0s 5ms/step - loss: 0.0984 - accuracy: 0.9702 - val_loss: 0.1839 - val_accuracy: 0.9419
Epoch 7/22
11/11 [==============================] - 0s 5ms/step - loss: 0.1141 - accuracy: 0.9603 - val_loss: 0.2210 - val_accuracy: 0.9302
Epoch 8/22
11/11 [=

In [27]:
eval_result = hypermodel.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_result)

5/5 [==============================] - 0s 1ms/step - loss: 0.2406 - accuracy: 0.9371
[test loss, test accuracy]: [0.24056287109851837, 0.9370629191398621]


In [28]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
cm = confusion_matrix(y_test,hypermodel.predict(x_test) > 0.5)
print(cm)


[[48  9]
 [ 0 86]]


In [29]:
best_model = tuner.get_best_hyperparameters(10)

In [30]:
print(best_model[0].values)

{'num_layers': 2, 'layer_0_width': 8, 'kernel_0': 'glorot_uniform', 'activation_0': 'tanh', 'dropout_0': 0.0, 'learning_rate': 0.01, 'batch_size': 8, 'layer_1_width': 128, 'kernel_1': 'glorot_uniform', 'activation_1': 'tanh', 'dropout_1': 0.2, 'layer_2_width': 64, 'kernel_2': 'glorot_uniform', 'activation_2': 'tanh', 'dropout_2': 0.1}
